In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#%run create_params.ipynb

In [4]:
"""
Coauthors: Michael Ainsworth
           Jayanta Dey
           Haoyin Xu
           Noga Mudrik
"""

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import cohen_kappa_score
import time
from toolbox import *

"""
Parameters
"""

num_classes = 10
reload_data = False  # indicator of whether to upload the data again

path_save = "metrics/cc18_all_parameters"
path_params = "metrics/dict_parameters"
path_train_val_test_indices = "metrics/dict_data_indices"
#with open(path_params + ".json", "r") as json_file:
#    dictionary_params = json.load(json_file)
f = open('metrics/dict_parameters.json');
dictionary_params = json.load(f)

f2 = open(path_train_val_test_indices + ".json")
dict_data_indices = json.load(f2)

f3 = open("metrics/varied_size_dict.json") #varied size are final indices for each model. it is dict -> dataset -> index sampline -> list
ss_inds_full = json.load(f3)

models_to_scale = {
    "RF": 0,
    "DN": 1,
    "GBDT": 0,
} 


#f3 = open(ss_inds_path + ".json")
#dict_data_indices = json.load(f2)
#with open(path_train_val_test_indices + ".json", "r") as json_file:
#    dict_data_indices = json.load(json_file)

path = path_save  # "metrics/cc18_all_parameters"
type_file = ".json"
dataset_indices_max = dictionary_params["dataset_indices_max"]
max_shape_to_run = dictionary_params["max_shape_to_run"]
file_type_to_load = ".json"

"""
Number of repetitions for each CV fold at each sample size
"""

reps = 1
shape_2_all_sample_sizes = dictionary_params["shape_2_all_sample_sizes"]
shape_2_evolution = dictionary_params["shape_2_evolution"]
n_splits = dictionary_params["shape_2_evolution"]

models_to_run = {"RF": 1, "DN": 1, "GBDT": 1}


"""
Import CC18 data and pretuned hyperparameters
"""

if reload_data or "dataset_name" not in locals():
    X_data_list, y_data_list, dataset_name = load_cc18()

"""
Upload best parameters
"""

#if "best_params_dict" not in locals():
f_best        = open(path+".json")
best_params_dict = json.load(f_best)

#train_indices = [i for i in range(dataset_indices_max)]

"""
Create empty arrays to index sample sizes, kappa values, and ece scores
"""

all_sample_sizes = np.zeros((dataset_indices_max, shape_2_all_sample_sizes))

"""
Empty arrays to index times into
"""

train_times = {
    model_name: None for model_name in dictionary_params["classifiers_names"]
}
test_times = {model_name: None for model_name in dictionary_params["classifiers_names"]}

train_test_times = {model_name: {} for model_name in best_params_dict.keys()}
#train_test_times = {
    #metric: {model_name: {} for model_name in best_params_dict.keys()}
    #for metric in ["cohen_kappa", "ece"]
#}


evolution_dict = {
    metric: {model_name: {} for model_name in best_params_dict.keys()}
    for metric in ["cohen_kappa", "ece"]
}

"""
For each dataset, train and predict for every sample size
Record outputs using Cohen's Kappa and ECE
"""

for dataset_index, dataset in enumerate(np.array(list(dict_data_indices.keys()),dtype = int)):
    if len(dict_data_indices[str(dataset)]) > 0:
        print("\n\nCurrent Dataset: ", dataset)
    
        X = X_data_list[dataset]
        y = y_data_list[dataset]
        
            # If data set has over 10000 samples, resample to contain 10000
        if X.shape[0] > max_shape_to_run:
            X, y = sample_large_datasets(X, y, max_shape_to_run)
            
        train_indices = dict_data_indices[str(dataset_index)]["train"]
        test_indices = dict_data_indices[str(dataset_index)]["test"]
    
    
        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]
    
        training_sample_sizes = np.geomspace(
            len(np.unique(y_train)) * 5, X_train.shape[0], num=8, dtype=int
        )
    
        #ss_inds = random_sample_all[dataset]
        #random_sample_new(
        #    X_train, y_train, training_sample_sizes
        #)
    
        # Iterate through each sample size per dataset
        try:
            ss_inds = ss_inds_full[dataset_index]
        except:
            ss_inds = ss_inds_full[str(dataset_index)]
        for sample_size_index, real_sample_size in enumerate(training_sample_sizes):
            real_sample_size  = int(real_sample_size)
            cohen_ece_results_dict = {metric: {} for metric in ["cohen_kappa", "ece"]}
            train_test_times_cur = {
                model_name: np.zeros((reps)) for model_name in best_params_dict.keys()
            }
            
            X_train_new = X_train[np.array(ss_inds[sample_size_index]).astype(int)]
            y_train_new = y_train[np.array(ss_inds[sample_size_index]).astype(int)]
    
            # Repeat for number of repetitions, averaging results
            for model_name, model_best_params in best_params_dict.items():
                if models_to_run[model_name]:
                    if models_to_scale[model_name ]:
                        scaler = StandardScaler()
                        scaler.fit(X_train_new)
                        X_train_new = scaler.transform(X_train_new)
        
                    if dataset not in train_test_times[model_name].keys():
                        train_test_times[model_name][dataset] = {}
                        train_test_times[model_name][dataset] = {}
    
                    try:
                        model = model_define(model_name, best_params_dict, dataset_index, sample_size_index)
    
                        start_time = time.perf_counter()
                        model.fit(X_train_new, y_train_new)
                        end_time = time.perf_counter()
                        train_time = end_time - start_time
    
                        predictions = model.predict(X_test)
                        predict_probas = model.predict_proba(X_test)
    
                        cohen_kappa = cohen_kappa_score(y_test, predictions)
                        cohen_ece_results_dict["cohen_kappa"][model_name] = cohen_kappa
                        ece = get_ece(predict_probas, predictions, y_test)
                        cohen_ece_results_dict["ece"][model_name] = ece
    
                        train_test_times_cur[model_name] = train_time
    
                    except ValueError:
                        print(model_name)
    
                    if dataset not in evolution_dict["cohen_kappa"][model_name].keys():
                        evolution_dict["cohen_kappa"][model_name][dataset] = {}
                        evolution_dict["ece"][model_name][dataset] = {}
                    if (
                        real_sample_size
                        not in evolution_dict["cohen_kappa"][model_name][dataset].keys()
                    ):
                        evolution_dict["cohen_kappa"][model_name][dataset][
                            real_sample_size
                        ] = []
                        evolution_dict["ece"][model_name][dataset][real_sample_size] = []
                        train_test_times[model_name][dataset][real_sample_size] = []
                        train_test_times[model_name][dataset][real_sample_size] = []
    
                    evolution_dict["cohen_kappa"][model_name][dataset][
                        real_sample_size
                    ].append(cohen_ece_results_dict["cohen_kappa"][model_name])
                    
                    evolution_dict["ece"][model_name][dataset][real_sample_size].append(
                        cohen_ece_results_dict["ece"][model_name]
                    )
                    train_test_times[model_name][dataset][real_sample_size].append(
                        train_test_times_cur[model_name]
                    )
    
                    # Changing the results to tuple enabling easier saving to json and ectacting the fata after that.
    
                    train_test_times[model_name][dataset][real_sample_size] = tuple(
                        train_test_times[model_name][dataset][real_sample_size]
                    )
    
                    evolution_dict["cohen_kappa"][model_name][dataset][
                        real_sample_size
                    ] = tuple(
                        evolution_dict["cohen_kappa"][model_name][dataset][real_sample_size]
                    )
    
                    evolution_dict["ece"][model_name][dataset][real_sample_size] = tuple(
                        evolution_dict["ece"][model_name][dataset][real_sample_size]
                    )
    
        # Record sample sizes used
        all_sample_sizes[dataset_index][:] = np.array(training_sample_sizes)

new_dict = {}
for key_met in evolution_dict.keys():
    new_dict[key_met] = mod_dict(evolution_dict[key_met], tuple)
#new_dict_times = mod_dict(train_test_times, tuple)


"""
Save sample sizes and model results in json files
"""


save_methods = {"json": 1}
save_methods_rewrite = {"json": 1}
save_best_parameters(
    save_methods,
    save_methods_rewrite,
    "metrics/cc18_sample_sizes.json",
    all_sample_sizes.tolist(),
)

save_best_parameters(
    save_methods, save_methods_rewrite, "results/cc18_kappa_and_ece", new_dict
)
save_best_parameters(
    save_methods, save_methods_rewrite, "results/cc18_training_times", train_test_times
)




Current Dataset:  0
Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 2.37175 |  0:00:00s
epoch 1  | loss: 1.46474 |  0:00:00s
epoch 2  | loss: 1.45166 |  0:00:00s
epoch 3  | loss: 0.90884 |  0:00:00s
epoch 4  | loss: 0.86081 |  0:00:00s
epoch 5  | loss: 1.6017  |  0:00:00s
epoch 6  | loss: 1.02068 |  0:00:00s
epoch 7  | loss: 1.4734  |  0:00:00s
epoch 8  | loss: 0.97165 |  0:00:00s
epoch 9  | loss: 0.72757 |  0:00:00s
epoch 10 | loss: 0.95151 |  0:00:00s
epoch 11 | loss: 1.01741 |  0:00:00s
epoch 12 | loss: 1.08218 |  0:00:01s
epoch 13 | loss: 0.68702 |  0:00:01s
epoch 14 | loss: 0.8023  |  0:00:01s
epoch 15 | loss: 0.87479 |  0:00:01s
epoch 16 | loss: 0.86579 |  0:00:01s
epoch 17 | loss: 0.9773  |  0:00:01s
epoch 18 | loss: 0.50087 |  0:00:01s
epoch 19 | loss: 0.34849 |  0:00:01s
epoch 20 | loss: 0.46568 |  0:00:01s
epoch 21 | loss: 0.50124 |  0:00:01s
epoch 22 | loss: 0.53685 |  0:00:01s
epoch 23 | loss: 0.50596 |  0:00:01s

C:\Users\noga mudrik\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 1.09899 |  0:00:00s
epoch 1  | loss: 1.27916 |  0:00:00s
epoch 2  | loss: 1.10794 |  0:00:00s
epoch 3  | loss: 0.76895 |  0:00:00s
epoch 4  | loss: 0.77077 |  0:00:00s
epoch 5  | loss: 0.91055 |  0:00:00s
epoch 6  | loss: 0.60117 |  0:00:00s
epoch 7  | loss: 0.81634 |  0:00:00s
epoch 8  | loss: 0.4935  |  0:00:00s
epoch 9  | loss: 0.59267 |  0:00:00s
epoch 10 | loss: 0.33696 |  0:00:00s
epoch 11 | loss: 0.60706 |  0:00:00s
epoch 12 | loss: 0.30982 |  0:00:00s
epoch 13 | loss: 0.3875  |  0:00:00s
epoch 14 | loss: 0.3735  |  0:00:00s
epoch 15 | loss: 0.39555 |  0:00:00s
epoch 16 | loss: 0.27403 |  0:00:00s
epoch 17 | loss: 0.39336 |  0:00:00s
epoch 18 | loss: 0.4018  |  0:00:00s
epoch 19 | loss: 0.26368 |  0:00:00s
epoch 20 | loss: 0.3045  |  0:00:00s
epoch 21 | loss: 0.38582 |  0:00:00s
epoch 22 | loss: 0.22374 |  0:00:00s
epoch 23 | loss: 0.30924 |  0:00:01s
epoch 24 | loss: 0.45

C:\Users\noga mudrik\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 0.96033 |  0:00:00s
epoch 1  | loss: 1.50268 |  0:00:00s
epoch 2  | loss: 1.08284 |  0:00:00s
epoch 3  | loss: 1.29756 |  0:00:00s
epoch 4  | loss: 1.19835 |  0:00:00s
epoch 5  | loss: 1.35281 |  0:00:00s
epoch 6  | loss: 1.29208 |  0:00:00s
epoch 7  | loss: 1.551   |  0:00:00s
epoch 8  | loss: 1.43929 |  0:00:00s
epoch 9  | loss: 1.28527 |  0:00:00s
epoch 10 | loss: 1.14703 |  0:00:00s
epoch 11 | loss: 1.21114 |  0:00:00s
epoch 12 | loss: 1.06401 |  0:00:00s
epoch 13 | loss: 1.03252 |  0:00:00s
epoch 14 | loss: 0.91138 |  0:00:00s
epoch 15 | loss: 0.8598  |  0:00:00s
epoch 16 | loss: 0.69804 |  0:00:00s
epoch 17 | loss: 0.69556 |  0:00:01s
epoch 18 | loss: 0.94139 |  0:00:01s
epoch 19 | loss: 0.65413 |  0:00:01s
epoch 20 | loss: 0.4347  |  0:00:01s
epoch 21 | loss: 0.53832 |  0:00:01s
epoch 22 | loss: 0.5373  |  0:00:01s
epoch 23 | loss: 0.56489 |  0:00:01s
epoch 24 | loss: 0.59

...

C:\Users\noga mudrik\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 0.84024 |  0:00:00s
epoch 1  | loss: 0.69435 |  0:00:00s
epoch 2  | loss: 0.68492 |  0:00:00s
epoch 3  | loss: 0.65718 |  0:00:00s
epoch 4  | loss: 0.63858 |  0:00:00s
epoch 5  | loss: 0.63143 |  0:00:00s
epoch 6  | loss: 0.61764 |  0:00:00s
epoch 7  | loss: 0.59993 |  0:00:00s
epoch 8  | loss: 0.58261 |  0:00:00s
epoch 9  | loss: 0.55777 |  0:00:00s
epoch 10 | loss: 0.54863 |  0:00:00s
epoch 11 | loss: 0.53419 |  0:00:00s
epoch 12 | loss: 0.51024 |  0:00:00s
epoch 13 | loss: 0.50221 |  0:00:00s
epoch 14 | loss: 0.45094 |  0:00:01s
epoch 15 | loss: 0.43925 |  0:00:01s
epoch 16 | loss: 0.41772 |  0:00:01s
epoch 17 | loss: 0.41244 |  0:00:01s
epoch 18 | loss: 0.3997  |  0:00:01s
epoch 19 | loss: 0.33797 |  0:00:01s
epoch 20 | loss: 0.34652 |  0:00:01s
epoch 21 | loss: 0.30844 |  0:00:01s
epoch 22 | loss: 0.30946 |  0:00:01s
epoch 23 | loss: 0.26453 |  0:00:01s
epoch 24 | loss: 0.28

C:\Users\noga mudrik\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:34:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


TypeError: keys must be str, int, float, bool or None, not int32

In [2]:
not_pyplot = True
import cc18_kappa_ece_times # main_functions
from importlib import reload  
cc18_hyperparameter = reload(cc18_hyperparameter)
from cc18_hyperparameter import * 




Current Dataset:  0
Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 2.37175 |  0:00:00s
epoch 1  | loss: 1.46474 |  0:00:00s
epoch 2  | loss: 1.45166 |  0:00:00s
epoch 3  | loss: 0.90884 |  0:00:00s
epoch 4  | loss: 0.86081 |  0:00:00s
epoch 5  | loss: 1.6017  |  0:00:00s
epoch 6  | loss: 1.02068 |  0:00:00s
epoch 7  | loss: 1.4734  |  0:00:00s
epoch 8  | loss: 0.97165 |  0:00:01s
epoch 9  | loss: 0.72757 |  0:00:01s
epoch 10 | loss: 0.95151 |  0:00:01s
epoch 11 | loss: 1.01741 |  0:00:01s
epoch 12 | loss: 1.08218 |  0:00:01s
epoch 13 | loss: 0.68702 |  0:00:01s
epoch 14 | loss: 0.8023  |  0:00:01s
epoch 15 | loss: 0.87479 |  0:00:01s
epoch 16 | loss: 0.86579 |  0:00:01s
epoch 17 | loss: 0.9773  |  0:00:01s
epoch 18 | loss: 0.50087 |  0:00:02s
epoch 19 | loss: 0.34849 |  0:00:02s
epoch 20 | loss: 0.46568 |  0:00:02s
epoch 21 | loss: 0.50124 |  0:00:02s
epoch 22 | loss: 0.53685 |  0:00:02s
epoch 23 | loss: 0.50596 |  0:00:02s

C:\Users\noga mudrik\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:45:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 1.09899 |  0:00:00s
epoch 1  | loss: 1.27916 |  0:00:00s
epoch 2  | loss: 1.10794 |  0:00:00s
epoch 3  | loss: 0.76895 |  0:00:00s
epoch 4  | loss: 0.77077 |  0:00:00s
epoch 5  | loss: 0.91055 |  0:00:00s
epoch 6  | loss: 0.60117 |  0:00:00s
epoch 7  | loss: 0.81634 |  0:00:00s
epoch 8  | loss: 0.4935  |  0:00:00s
epoch 9  | loss: 0.59267 |  0:00:00s
epoch 10 | loss: 0.33696 |  0:00:00s
epoch 11 | loss: 0.60706 |  0:00:00s
epoch 12 | loss: 0.30982 |  0:00:00s
epoch 13 | loss: 0.3875  |  0:00:00s
epoch 14 | loss: 0.3735  |  0:00:00s
epoch 15 | loss: 0.39555 |  0:00:00s
epoch 1

C:\Users\noga mudrik\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 0.96033 |  0:00:00s
epoch 1  | loss: 1.50268 |  0:00:00s
epoch 2  | loss: 1.08284 |  0:00:00s
epoch 3  | loss: 1.29756 |  0:00:00s
epoch 4  | loss: 1.19835 |  0:00:00s
epoch 5  | loss: 1.35281 |  0:00:00s
epoch 6  | loss: 1.29208 |  0:00:00s
epoch 7  | loss: 1.551   |  0:00:00s
epoch 8  | loss: 1.43929 |  0:00:00s
epoch 9  | loss: 1.28527 |  0:00:00s
epoch 10 | loss: 1.14703 |  0:00:00s
epoch 11 | loss: 1.21114 |  0:00:00s
epoch 12 | loss: 1.06401 |  0:00:00s
epoch 13 | loss: 1.03252 |  0:00:00s
epoch 14 | loss: 0.91138 |  0:00:00s
epoch 15 | loss: 0.8598  |  0:00:00s
epoch 16 | loss: 0.69804 |  0:00:00s
epoch 17 | loss: 0.69556 |  0:00:00s
epoch 18 | loss: 0.94139 |  0:00:00s
epoch 19 | loss: 0.65413 |  0:00:00s
epoch 20 | loss: 0.4347  |  0:00:01s
epoch 21 | loss: 0.53832 |  0:00:01s
epoch 22 | loss: 0.5373  |  0:00:01s
epoch 23 | loss: 0.56489 |  0:00:01s
epoch 24 | loss: 0.59

...

[13:45:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 0.99937 |  0:00:00s
epoch 1  | loss: 0.90945 |  0:00:00s
epoch 2  | loss: 0.75641 |  0:00:00s
epoch 3  | loss: 0.7016  |  0:00:00s
epoch 4  | loss: 0.69314 |  0:00:00s
epoch 5  | loss: 0.68586 |  0:00:00s
epoch 6  | loss: 0.67249 |  0:00:00s
epoch 7  | loss: 0.67692 |  0:00:00s
epoch 8  | loss: 0.6602  |  0:00:00s
epoch 9  | loss: 0.61932 |  0:00:00s
epoch 10 | loss: 0.62992 |  0:00:00s
epoch 11 | loss: 0.59722 |  0:00:00s
epoch 12 | loss: 0.54761 |  0:00:01s
epoch 13 | loss: 0.5966  |  0:00:01s
epoch 14 | loss: 0.48866 |  0:00:01s
epoch 15 | loss: 0.54141 |  0:00:01s
epoch 1

C:\Users\noga mudrik\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 0.84024 |  0:00:00s
epoch 1  | loss: 0.69435 |  0:00:00s
epoch 2  | loss: 0.68492 |  0:00:00s
epoch 3  | loss: 0.65718 |  0:00:00s
epoch 4  | loss: 0.63858 |  0:00:00s
epoch 5  | loss: 0.63143 |  0:00:00s
epoch 6  | loss: 0.61764 |  0:00:00s
epoch 7  | loss: 0.59993 |  0:00:00s
epoch 8  | loss: 0.58261 |  0:00:00s
epoch 9  | loss: 0.55777 |  0:00:00s
epoch 10 | loss: 0.54863 |  0:00:00s
epoch 11 | loss: 0.53419 |  0:00:00s
epoch 12 | loss: 0.51024 |  0:00:00s
epoch 13 | loss: 0.50221 |  0:00:00s
epoch 14 | loss: 0.45094 |  0:00:01s
epoch 15 | loss: 0.43925 |  0:00:01s
epoch 16 | loss: 0.41772 |  0:00:01s
epoch 17 | loss: 0.41244 |  0:00:01s
epoch 18 | loss: 0.3997  |  0:00:01s
epoch 19 | loss: 0.33797 |  0:00:01s
epoch 20 | loss: 0.34652 |  0:00:01s
epoch 21 | loss: 0.30844 |  0:00:01s
epoch 22 | loss: 0.30946 |  0:00:01s
epoch 23 | loss: 0.26453 |  0:00:01s
epoch 24 | loss: 0.28

C:\Users\noga mudrik\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


TypeError: keys must be str, int, float, bool or None, not int32

In [6]:
new_dict

{'cohen_kappa': {'RF': {0: {10: (0.2739516180506798,),
    17: (0.3171154958544894,),
    30: (0.7526648369183635,),
    53: (0.6433434926399066,),
    93: (0.7921596214107566,),
    163: (0.855444908448442,),
    285: (0.9198769309659637,),
    500: (0.9679507723863855,)}},
  'DN': {0: {10: (0.17047517686095282,),
    17: (-0.024778219639033372,),
    30: (0.16414839580788276,),
    53: (0.009587855383181187,),
    93: (0.43564745141660677,),
    163: (0.05793302313388826,),
    285: (0.48323751695626904,),
    500: (0.17066281284897844,)}},
  'GBDT': {0: {10: (0.0,),
    17: (0.0,),
    30: (0.0,),
    53: (0.09066860657363918,),
    93: (0.0,),
    163: (0.0008060724121716767,),
    285: (0.13096735747864674,),
    500: (0.13096735747864674,)}}},
 'ece': {'RF': {0: {10: (0.12503333333333333,),
    17: (0.12144000000000002,),
    30: (0.18308000000000002,),
    53: (0.07200000000000002,),
    93: (0.08779579365079368,),
    163: (0.04039999999999999,),
    285: (0.08666301004999788,)